In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os

In [2]:
sampleSize=16000
sample_rate=16000
quantization_channels=256
#dilations=[2**i for i in range(8)]*20
#"residualDim=32
dilations=[2**i for i in range(9)]*5
residualDim=128
skipDim=512
filterSize=3
resumefile='trypadmodel'
continueTrain=False
pad = np.sum(dilations)
lossrecord=[]
#pad=0

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
def mu_law_encode(audio, quantization_channels=quantization_channels,forX=False):
    '''Quantizes waveform amplitudes.'''
    mu = (quantization_channels - 1)*1.0
    # Perform mu-law companding transformation (ITU-T, 1988).
    # Minimum operation is here to deal with rare large amplitudes caused
    # by resampling.
    safe_audio_abs = np.minimum(np.abs(audio), 1.0)
    magnitude = np.log1p(mu * safe_audio_abs) / np.log1p(mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    if(forX):return signal
    return ((signal + 1) / 2 * mu + 0.5).astype(int)
def mu_law_decode(output, quantization_channels=quantization_channels):
    '''Recovers waveform from quantized values.'''
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    # Perform inverse of mu-law transformation.
    magnitude = (1 / mu) * ((1 + mu)**np.abs(signal) - 1)
    return np.sign(signal) * magnitude
def onehot(a,mu=quantization_channels):
    b = np.zeros((a.shape[0], mu))
    b[np.arange(a.shape[0]), a] = 1
    return b

In [6]:
def readAudio(name):
    audio0, samplerate = sf.read(name, dtype='float32')
    return librosa.resample(audio0.T, samplerate, sample_rate).reshape(-1)
p=['./vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav',
   './vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav','./vsCorpus/pred_mix.wav']
xtrain,ytrain,xval,yval,xtest=readAudio(p[0]),readAudio(p[1]),readAudio(p[2]),readAudio(p[3]),readAudio(p[4])
assert((xtrain==xval).all())
assert((ytrain==yval).all())
assert((xtrain != ytrain).any())

In [7]:
ytrain,yval=mu_law_encode(ytrain),mu_law_encode(yval)
xtrain,xval,xtest=mu_law_encode(xtrain,forX=True),mu_law_encode(xval,forX=True),mu_law_encode(xtest,forX=True)
assert (xtrain.max()<=2 and ytrain.max() >= 5)

In [8]:
'''xmean,xstd = xtrain.mean(),xtrain.std()
xtrain=(xtrain-xmean)/xstd
xval=(xval-xmean)/xstd
xtest=(xtest-xmean)/xstd'''

'xmean,xstd = xtrain.mean(),xtrain.std()\nxtrain=(xtrain-xmean)/xstd\nxval=(xval-xmean)/xstd\nxtest=(xtest-xmean)/xstd'

In [9]:
xtrain=np.pad(xtrain, (pad, pad), 'constant')
xval=np.pad(xval, (pad, pad), 'constant')
xtest=np.pad(xtest, (pad, pad), 'constant')
yval=np.pad(yval, (pad, pad), 'constant')
ytrain=np.pad(ytrain, (pad, pad), 'constant')

In [10]:
xtrain,xval,xtest=xtrain.reshape(1,1,-1),xval.reshape(1,1,-1),xtest.reshape(1,1,-1)
ytrain,yval=ytrain.reshape(1,-1),yval.reshape(1,-1)

In [11]:
xtrain,ytrain,xval,yval,xtest = torch.from_numpy(xtrain).type(torch.float32),\
                                torch.from_numpy(ytrain).type(torch.LongTensor),\
                                torch.from_numpy(xval).type(torch.float32),\
                                torch.from_numpy(yval).type(torch.LongTensor),\
                                torch.from_numpy(xtest).type(torch.float32)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        sd,qd,rd = skipDim,quantization_channels,residualDim
        self.causal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=3,padding=1)
        self.tanhconvs = nn.ModuleList()
        self.sigmoidconvs = nn.ModuleList()
        self.skipconvs = nn.ModuleList()
        self.denseconvs = nn.ModuleList()
        for i, d in enumerate(dilations):
            self.tanhconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=0,dilation=d))
            self.sigmoidconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=0,dilation=d))
            self.skipconvs.append(nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1))
            self.denseconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1))
        self.post1 = nn.Conv1d(in_channels=sd,out_channels=sd,kernel_size=1)
        self.post2 = nn.Conv1d(in_channels=sd,out_channels=qd,kernel_size=1)
        self.tanh,self.sigmoid = nn.Tanh(),nn.Sigmoid()

    def forward(self, x):
        finallen = x.shape[-1]-2*pad
        x = self.causal(x)
        for i, dilation in enumerate(dilations):
            xinput = x.clone()[:,:,dilation:-dilation]
            x1 = self.tanh(self.tanhconvs[i](x))
            x2 = self.sigmoid(self.sigmoidconvs[i](x))
            x = x1*x2
            cutlen = (x.shape[-1] - finallen)//2
            if(i == 0):skip_connections= (self.skipconvs[i](x)).narrow(2,int(cutlen),int(finallen))
            else :skip_connections += (self.skipconvs[i](x)).narrow(2,int(cutlen),int(finallen))
            x = self.denseconvs[i](x)
            x += xinput
        x = self.post2(F.relu(self.post1(F.relu(skip_connections))))
        return x

model = Net()#.cuda()
criterion = nn.CrossEntropyLoss()#.cuda()
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-5)

In [ ]:
if continueTrain:
    if os.path.isfile(resumefile):
        print("=> loading checkpoint '{}'".format(resumefile))
        checkpoint = torch.load(resumefile)
        start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resumefile, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resumefile))

In [15]:
def val():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,1000)
        np.random.shuffle(idx)
        data = xtrain[:,:,idx[0]-pad:pad+idx[0]+sampleSize].to(device)
        target = ytrain[:,idx[0]:idx[0]+sampleSize].to(device)
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
        val_loss = criterion(output, target).item()
    print(correct,'accurate')
    print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss,time.time()-startval_time))

def test():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        listofpred = []
        for ind in range(pad, xtest.shape[-1] - pad, sampleSize):
            output = model(xtest[:, :, ind - pad:ind + sampleSize + pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/notexte.wav', ans, sample_rate)

        listofpred=[]
        for ind in range(pad,xtrain.shape[-1]-pad,sampleSize):
            output = model(xtrain[:, :, ind-pad:ind+sampleSize+pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/notextr.wav', ans, sample_rate)
        print('stored done\n')
    
def train(epoch):
    model.train()
    idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,8000)
    np.random.shuffle(idx)
    for i, ind in enumerate(idx):
        start_time = time.time()
        data, target = xtrain[:,:,ind-pad:ind+sampleSize+pad].to(device), ytrain[:,ind:ind+sampleSize].to(device)
        output = model(data)
        loss = criterion(output, target)
        lossrecord.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
                epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
        if i % 100 == 0:
            with open("./lossRecord/trypadlossfile.txt", "w") as f:
                for s in lossrecord:
                    f.write(str(s) +"\n")
            print('write finish')
            state={'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()}
            torch.save(state, './model/'+resumefile)
    test()
    val()

In [ ]:
for epoch in range(100000):
    train(epoch)
    test()

Train Epoch: 0 [0/470 (0%)] Loss:4.341933: , (1.201 sec/step)
write finish
Train Epoch: 0 [1/470 (0%)] Loss:4.321697: , (1.201 sec/step)
Train Epoch: 0 [2/470 (0%)] Loss:4.207393: , (1.199 sec/step)
Train Epoch: 0 [3/470 (1%)] Loss:4.030066: , (1.199 sec/step)
Train Epoch: 0 [4/470 (1%)] Loss:3.854748: , (1.198 sec/step)
Train Epoch: 0 [5/470 (1%)] Loss:4.296246: , (1.198 sec/step)
Train Epoch: 0 [6/470 (1%)] Loss:4.222650: , (1.197 sec/step)
Train Epoch: 0 [7/470 (1%)] Loss:4.054540: , (1.196 sec/step)
Train Epoch: 0 [8/470 (2%)] Loss:4.152524: , (1.196 sec/step)
Train Epoch: 0 [9/470 (2%)] Loss:4.155514: , (1.197 sec/step)
Train Epoch: 0 [10/470 (2%)] Loss:4.004141: , (1.196 sec/step)
Train Epoch: 0 [11/470 (2%)] Loss:4.447809: , (1.194 sec/step)
Train Epoch: 0 [12/470 (3%)] Loss:4.078428: , (1.197 sec/step)
Train Epoch: 0 [13/470 (3%)] Loss:4.145298: , (1.195 sec/step)
Train Epoch: 0 [14/470 (3%)] Loss:4.256442: , (1.200 sec/step)
Train Epoch: 0 [15/470 (3%)] Loss:4.208318: , (1.193

Train Epoch: 0 [129/470 (27%)] Loss:4.347756: , (1.198 sec/step)
Train Epoch: 0 [130/470 (28%)] Loss:4.744690: , (1.197 sec/step)
Train Epoch: 0 [131/470 (28%)] Loss:3.954391: , (1.196 sec/step)
Train Epoch: 0 [132/470 (28%)] Loss:4.062417: , (1.201 sec/step)
Train Epoch: 0 [133/470 (28%)] Loss:4.699494: , (1.204 sec/step)
Train Epoch: 0 [134/470 (29%)] Loss:4.224154: , (1.196 sec/step)
Train Epoch: 0 [135/470 (29%)] Loss:4.302280: , (1.194 sec/step)
Train Epoch: 0 [136/470 (29%)] Loss:4.529491: , (1.194 sec/step)
Train Epoch: 0 [137/470 (29%)] Loss:4.063563: , (1.194 sec/step)
Train Epoch: 0 [138/470 (29%)] Loss:4.213327: , (1.194 sec/step)
Train Epoch: 0 [139/470 (30%)] Loss:4.370289: , (1.193 sec/step)
Train Epoch: 0 [140/470 (30%)] Loss:4.159316: , (1.193 sec/step)
Train Epoch: 0 [141/470 (30%)] Loss:4.097656: , (1.193 sec/step)
Train Epoch: 0 [142/470 (30%)] Loss:4.402174: , (1.193 sec/step)
Train Epoch: 0 [143/470 (30%)] Loss:4.281171: , (1.193 sec/step)
Train Epoch: 0 [144/470 (

Train Epoch: 0 [255/470 (54%)] Loss:4.265447: , (1.196 sec/step)
Train Epoch: 0 [256/470 (54%)] Loss:4.570208: , (1.195 sec/step)
Train Epoch: 0 [257/470 (55%)] Loss:4.088964: , (1.195 sec/step)
Train Epoch: 0 [258/470 (55%)] Loss:4.057946: , (1.194 sec/step)
Train Epoch: 0 [259/470 (55%)] Loss:4.176943: , (1.195 sec/step)
Train Epoch: 0 [260/470 (55%)] Loss:3.752998: , (1.196 sec/step)
Train Epoch: 0 [261/470 (56%)] Loss:4.024511: , (1.194 sec/step)
Train Epoch: 0 [262/470 (56%)] Loss:4.371039: , (1.194 sec/step)
Train Epoch: 0 [263/470 (56%)] Loss:4.680456: , (1.195 sec/step)
Train Epoch: 0 [264/470 (56%)] Loss:4.004265: , (1.195 sec/step)
Train Epoch: 0 [265/470 (56%)] Loss:4.213607: , (1.196 sec/step)
Train Epoch: 0 [266/470 (57%)] Loss:4.047861: , (1.196 sec/step)
Train Epoch: 0 [267/470 (57%)] Loss:3.946784: , (1.194 sec/step)
Train Epoch: 0 [268/470 (57%)] Loss:4.188454: , (1.195 sec/step)
Train Epoch: 0 [269/470 (57%)] Loss:4.025015: , (1.195 sec/step)
Train Epoch: 0 [270/470 (

Train Epoch: 0 [381/470 (81%)] Loss:4.136870: , (1.199 sec/step)
Train Epoch: 0 [382/470 (81%)] Loss:4.325212: , (1.199 sec/step)
Train Epoch: 0 [383/470 (81%)] Loss:4.173262: , (1.200 sec/step)
Train Epoch: 0 [384/470 (82%)] Loss:4.426903: , (1.200 sec/step)
Train Epoch: 0 [385/470 (82%)] Loss:3.941491: , (1.200 sec/step)
Train Epoch: 0 [386/470 (82%)] Loss:4.351096: , (1.200 sec/step)
Train Epoch: 0 [387/470 (82%)] Loss:4.585780: , (1.199 sec/step)
Train Epoch: 0 [388/470 (83%)] Loss:4.012942: , (1.200 sec/step)
Train Epoch: 0 [389/470 (83%)] Loss:4.291359: , (1.200 sec/step)
Train Epoch: 0 [390/470 (83%)] Loss:3.809860: , (1.201 sec/step)
Train Epoch: 0 [391/470 (83%)] Loss:4.073983: , (1.201 sec/step)
Train Epoch: 0 [392/470 (83%)] Loss:4.508218: , (1.201 sec/step)
Train Epoch: 0 [393/470 (84%)] Loss:4.336500: , (1.201 sec/step)
Train Epoch: 0 [394/470 (84%)] Loss:3.965216: , (1.200 sec/step)
Train Epoch: 0 [395/470 (84%)] Loss:4.259925: , (1.200 sec/step)
Train Epoch: 0 [396/470 (

Train Epoch: 1 [37/470 (8%)] Loss:4.717124: , (1.194 sec/step)
Train Epoch: 1 [38/470 (8%)] Loss:3.986355: , (1.193 sec/step)
Train Epoch: 1 [39/470 (8%)] Loss:4.342321: , (1.193 sec/step)
Train Epoch: 1 [40/470 (9%)] Loss:4.349516: , (1.193 sec/step)
Train Epoch: 1 [41/470 (9%)] Loss:4.127096: , (1.194 sec/step)
Train Epoch: 1 [42/470 (9%)] Loss:4.472117: , (1.193 sec/step)
Train Epoch: 1 [43/470 (9%)] Loss:4.225720: , (1.194 sec/step)
Train Epoch: 1 [44/470 (9%)] Loss:4.370316: , (1.192 sec/step)
Train Epoch: 1 [45/470 (10%)] Loss:4.271725: , (1.194 sec/step)
Train Epoch: 1 [46/470 (10%)] Loss:4.273636: , (1.193 sec/step)
Train Epoch: 1 [47/470 (10%)] Loss:3.880467: , (1.195 sec/step)
Train Epoch: 1 [48/470 (10%)] Loss:4.345844: , (1.194 sec/step)
Train Epoch: 1 [49/470 (10%)] Loss:4.312946: , (1.195 sec/step)
Train Epoch: 1 [50/470 (11%)] Loss:4.027793: , (1.196 sec/step)
Train Epoch: 1 [51/470 (11%)] Loss:3.820653: , (1.196 sec/step)
Train Epoch: 1 [52/470 (11%)] Loss:3.954380: , (

...

Train Epoch: 50 [91/470 (19%)] Loss:2.357209: , (1.197 sec/step)
Train Epoch: 50 [92/470 (20%)] Loss:3.191988: , (1.197 sec/step)
Train Epoch: 50 [93/470 (20%)] Loss:2.373844: , (1.197 sec/step)
Train Epoch: 50 [94/470 (20%)] Loss:2.092418: , (1.198 sec/step)
Train Epoch: 50 [95/470 (20%)] Loss:2.386151: , (1.198 sec/step)
Train Epoch: 50 [96/470 (20%)] Loss:2.919730: , (1.199 sec/step)
Train Epoch: 50 [97/470 (21%)] Loss:2.363140: , (1.199 sec/step)
Train Epoch: 50 [98/470 (21%)] Loss:2.299977: , (1.198 sec/step)
Train Epoch: 50 [99/470 (21%)] Loss:3.147824: , (1.197 sec/step)
Train Epoch: 50 [100/470 (21%)] Loss:2.225197: , (1.199 sec/step)
write finish
Train Epoch: 50 [101/470 (21%)] Loss:2.294425: , (1.198 sec/step)
Train Epoch: 50 [102/470 (22%)] Loss:2.257690: , (1.198 sec/step)
Train Epoch: 50 [103/470 (22%)] Loss:2.526130: , (1.198 sec/step)
Train Epoch: 50 [104/470 (22%)] Loss:2.843565: , (1.197 sec/step)
Train Epoch: 50 [105/470 (22%)] Loss:2.600883: , (1.198 sec/step)
Train 

Train Epoch: 50 [215/470 (46%)] Loss:2.231292: , (1.196 sec/step)
Train Epoch: 50 [216/470 (46%)] Loss:2.376177: , (1.197 sec/step)
Train Epoch: 50 [217/470 (46%)] Loss:2.250104: , (1.196 sec/step)
Train Epoch: 50 [218/470 (46%)] Loss:1.857661: , (1.197 sec/step)
Train Epoch: 50 [219/470 (47%)] Loss:2.374482: , (1.197 sec/step)
Train Epoch: 50 [220/470 (47%)] Loss:2.990928: , (1.197 sec/step)
Train Epoch: 50 [221/470 (47%)] Loss:2.039023: , (1.197 sec/step)
Train Epoch: 50 [222/470 (47%)] Loss:2.402304: , (1.197 sec/step)
Train Epoch: 50 [223/470 (47%)] Loss:1.795668: , (1.197 sec/step)
Train Epoch: 50 [224/470 (48%)] Loss:2.443270: , (1.196 sec/step)
Train Epoch: 50 [225/470 (48%)] Loss:2.270430: , (1.197 sec/step)
Train Epoch: 50 [226/470 (48%)] Loss:2.167653: , (1.199 sec/step)
Train Epoch: 50 [227/470 (48%)] Loss:2.014937: , (1.197 sec/step)
Train Epoch: 50 [228/470 (49%)] Loss:2.048087: , (1.197 sec/step)
Train Epoch: 50 [229/470 (49%)] Loss:2.464362: , (1.197 sec/step)
Train Epoc

Train Epoch: 50 [339/470 (72%)] Loss:3.165793: , (1.198 sec/step)
Train Epoch: 50 [340/470 (72%)] Loss:3.123488: , (1.197 sec/step)
Train Epoch: 50 [341/470 (73%)] Loss:2.494263: , (1.197 sec/step)
Train Epoch: 50 [342/470 (73%)] Loss:1.935878: , (1.206 sec/step)
Train Epoch: 50 [343/470 (73%)] Loss:2.789383: , (1.196 sec/step)
Train Epoch: 50 [344/470 (73%)] Loss:2.327103: , (1.197 sec/step)
Train Epoch: 50 [345/470 (73%)] Loss:2.740468: , (1.196 sec/step)
Train Epoch: 50 [346/470 (74%)] Loss:2.443315: , (1.195 sec/step)
Train Epoch: 50 [347/470 (74%)] Loss:2.203702: , (1.196 sec/step)
Train Epoch: 50 [348/470 (74%)] Loss:3.596414: , (1.195 sec/step)
Train Epoch: 50 [349/470 (74%)] Loss:3.370260: , (1.196 sec/step)
Train Epoch: 50 [350/470 (74%)] Loss:2.778844: , (1.196 sec/step)
Train Epoch: 50 [351/470 (75%)] Loss:2.732866: , (1.195 sec/step)
Train Epoch: 50 [352/470 (75%)] Loss:2.557390: , (1.196 sec/step)
Train Epoch: 50 [353/470 (75%)] Loss:3.573767: , (1.195 sec/step)
Train Epoc

Train Epoch: 50 [463/470 (99%)] Loss:2.091201: , (1.201 sec/step)
Train Epoch: 50 [464/470 (99%)] Loss:2.766657: , (1.200 sec/step)
Train Epoch: 50 [465/470 (99%)] Loss:2.161091: , (1.200 sec/step)
Train Epoch: 50 [466/470 (99%)] Loss:2.197581: , (1.200 sec/step)
Train Epoch: 50 [467/470 (99%)] Loss:2.390954: , (1.201 sec/step)
Train Epoch: 50 [468/470 (100%)] Loss:2.156658: , (1.200 sec/step)
Train Epoch: 50 [469/470 (100%)] Loss:2.944313: , (1.198 sec/step)
stored done

0.3246875 accurate

val set:loss2.4766:, (0.364 sec/step)

stored done

Train Epoch: 51 [0/470 (0%)] Loss:2.236990: , (1.194 sec/step)
write finish
Train Epoch: 51 [1/470 (0%)] Loss:1.856307: , (1.195 sec/step)
Train Epoch: 51 [2/470 (0%)] Loss:1.896476: , (1.196 sec/step)
Train Epoch: 51 [3/470 (1%)] Loss:2.926199: , (1.194 sec/step)
Train Epoch: 51 [4/470 (1%)] Loss:2.127220: , (1.196 sec/step)
Train Epoch: 51 [5/470 (1%)] Loss:2.255368: , (1.195 sec/step)
Train Epoch: 51 [6/470 (1%)] Loss:2.221799: , (1.194 sec/ste

Train Epoch: 51 [118/470 (25%)] Loss:2.258712: , (1.197 sec/step)
Train Epoch: 51 [119/470 (25%)] Loss:2.526684: , (1.197 sec/step)
Train Epoch: 51 [120/470 (26%)] Loss:2.615652: , (1.196 sec/step)
Train Epoch: 51 [121/470 (26%)] Loss:2.225200: , (1.197 sec/step)
Train Epoch: 51 [122/470 (26%)] Loss:3.187238: , (1.195 sec/step)
Train Epoch: 51 [123/470 (26%)] Loss:2.786849: , (1.197 sec/step)
Train Epoch: 51 [124/470 (26%)] Loss:2.386204: , (1.195 sec/step)
Train Epoch: 51 [125/470 (27%)] Loss:2.182469: , (1.194 sec/step)
Train Epoch: 51 [126/470 (27%)] Loss:2.117226: , (1.196 sec/step)
Train Epoch: 51 [127/470 (27%)] Loss:2.490390: , (1.194 sec/step)
Train Epoch: 51 [128/470 (27%)] Loss:2.319702: , (1.194 sec/step)
Train Epoch: 51 [129/470 (27%)] Loss:2.592889: , (1.195 sec/step)
Train Epoch: 51 [130/470 (28%)] Loss:2.807010: , (1.194 sec/step)
Train Epoch: 51 [131/470 (28%)] Loss:2.704062: , (1.192 sec/step)
Train Epoch: 51 [132/470 (28%)] Loss:2.006600: , (1.191 sec/step)
Train Epoc

In [ ]:
#torch.save(model, 'loss2.5~3.5step10h_repeat5*2**9resu32sample50000')